In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
import pandas

In [15]:
from pymongo import MongoClient
from pprint import pprint

In [163]:
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0',
}
first_pg_hh='https://hh.ru/vacancies/shveya'

In [164]:
def total_parsing_hh(first_pg, headers, result=[]):

    response = requests.get(first_pg, headers=headers) 
    
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    soup = bs(response.content, 'html.parser')
    tag_main_div=soup.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in tag_main_div:
        result.append(vac_parsing(vacancy))
    
    link_next_page = soup.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
        link_next_page =  'https://hh.ru' + link_next_page['href']
        
    else:
        print('Парсинг завершен')
        return result
    result=total_parsing_hh(link_next_page, headers,result)                        
    return result
    

In [165]:
hh_vcncy_list=[]
def salary_split(tag_span_salary, vac_min_salary=None, vac_max_salary=None, currency=None):
    sleep(0.1)
    tag_span_salary=tag_span_salary.text.replace('\u202f','').split()   
    if tag_span_salary[1]=='–':
        vac_max_salary=int(tag_span_salary[2])
        vac_min_salary=int(tag_span_salary[0])    
    elif  tag_span_salary[0]=='до':
        vac_max_salary=int(tag_span_salary[1])        
    elif tag_span_salary[0]=='от':
        vac_min_salary=int(tag_span_salary[1])   
    currency=tag_span_salary[-1]
        
    
    return vac_min_salary, vac_max_salary, currency
def vac_parsing(tag_main_div):
    sleep(0.1)

    tag_a_name=tag_main_div.find('a').text.strip()

    tag_span_salary=tag_main_div.find('span', {'class', 'bloko-header-section-3'})
    if tag_span_salary:
          vac_min_salary, vac_max_salary, currency=salary_split(tag_span_salary)
    else:
        vac_min_salary, vac_max_salary, currency=None,None,None
    tag_a_vcnc_link=tag_main_div.find('a')['href']
    
    return {
        'name': tag_a_name,
        'min_salary': vac_min_salary,
        'max_salary': vac_max_salary,
        'currency': currency,
        'vac_link': tag_a_vcnc_link,
        'site':'https://hh.ru'
        }

In [166]:
result=total_parsing_hh(first_pg_hh,headers)

Парсинг завершен


In [167]:
len(result)

391

In [168]:
data_frame_hh = pandas.DataFrame(result) 

data_frame_hh.to_csv('hh.csv') 

Добавляем новые вакансии в Базу Данных с проверкой.

In [169]:
client = MongoClient('mongodb://127.0.0.1:27017/')
print(client.list_database_names())

['admin', 'config', 'hh_vac', 'local']


In [170]:
db = client.hh_vac
print(db.list_collection_names())

['vac']


In [171]:
vacancyes = db.vac.find()
list(vacancyes)[3]

{'_id': ObjectId('63209b1dc468a33dff3168f3'),
 '': '3',
 'name': 'Швея',
 'min_salary': 30000,
 'max_salary': 100000,
 'currency': 'руб.',
 'vac_link': 'https://hh.ru/vacancy/69621662?query=%D1%88%D0%B2%D0%B5%D1%8F',
 'site': 'https://hh.ru'}

In [66]:
def nw_vac_hh(new_data):
    count=0
    for data in new_data:
        if db.vac.find_one({ 'vac_link': data['vac_link']}):
            print(' уже есть в базе', (data['name']))
        else:
            db.vac.insert_one(data)
            print(' добавлена',(data['name']))
            count+=1
    print(count)

In [172]:
result_add=total_parsing_hh('https://hh.ru/search/vacancy?area=1&professional_role=49&search_field=name&search_field=company_name&search_field=description&text=%D0%BF%D0%BE%D1%80%D1%82%D0%BD%D0%BE%D0%B9',headers)
nw_vac_hh(result_add)

Парсинг завершен
 уже есть в базе Портной, швея на производство одежды
 уже есть в базе Швея/портной-универсал
 уже есть в базе Швея-портной
 уже есть в базе Швея
 уже есть в базе Швея ученик
 уже есть в базе Швея
 уже есть в базе Швея
 уже есть в базе Швея-портной
 уже есть в базе Швея
 уже есть в базе Швея
 уже есть в базе Швея-портной
 уже есть в базе Швея
 уже есть в базе Швея
 уже есть в базе Швея
 уже есть в базе Швея-портной (при необходимости с предоставлением жилья)
 уже есть в базе Швея
 уже есть в базе Швея / портной (женская одежда)
 уже есть в базе Швея в швейный цех
 уже есть в базе Швея по ремонту одежды (подработка).
 уже есть в базе Швея
 уже есть в базе Швея
 уже есть в базе Швея-портной
 уже есть в базе Швея - портной по ремонтам или на индивидуальный пошив в ателье
 уже есть в базе Швея (работа или подработка)
 уже есть в базе Швея
 уже есть в базе Швея-портной
 уже есть в базе Швея-портной
 уже есть в базе Швея
 уже есть в базе Швея
 уже есть в базе Швея-портной
 у

 уже есть в базе Швея-портной
 уже есть в базе Швея
 уже есть в базе Швея-портной (м. Семёновская)
 уже есть в базе Швея-портной
 уже есть в базе Швея-ручница (ручник) ОТК
 уже есть в базе Швея-лаборант
 уже есть в базе Швея по пошиву штор
 уже есть в базе Швея-портной
 уже есть в базе Швеи на производство спецодежды (Москва)
 уже есть в базе Швея универсал
 уже есть в базе Швеи на производство женской одежды (Москва)
 уже есть в базе Швея по пошиву покрывал
 уже есть в базе Швея-портной
 уже есть в базе Швея-портной
 уже есть в базе Швея по пошиву штор
 уже есть в базе Швея по пошиву женской одежды
 уже есть в базе Швея-портной (м. Выхино)
 уже есть в базе Лаборант/швея
 уже есть в базе Швея-портной
 уже есть в базе Швея-ручница/ручник в швейный цех
 уже есть в базе Швея
 уже есть в базе Швея в швейный цех
 уже есть в базе Швея-портной (ТРЦ "Саларис")
 уже есть в базе Швея-портной
 уже есть в базе Швея 4 разряда-6 разряда
 уже есть в базе Швея по пошиву штор и домашнего текстиля
 уже 

 Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [184]:
def chek_salary(max_salary):
    count=0
    salary=db.vac.find({'min_salary':{'$gt':max_salary},'max_salary':{'$gt':max_salary}})
    for i in salary:
        count+=1
        print(i['name'], 'может получать зп больше', max_salary)
    return count   

In [185]:
chek_salary(85000)

Швея / портной (женская одежда) может получать зп больше 85000
Швея-портной может получать зп больше 85000
Швея - портной по ремонтам или на индивидуальный пошив в ателье может получать зп больше 85000
Швея может получать зп больше 85000
Швея может получать зп больше 85000
Швея на производство по пошиву пледов, термосумок и шоперов может получать зп больше 85000
Швея на производство постельного белья (Москва) может получать зп больше 85000
Швея на производство постельного белья (Москва) может получать зп больше 85000
Швея может получать зп больше 85000
Швея в швейный цех текстиля может получать зп больше 85000
Швея на производство для пошива женской одежды (Москва) может получать зп больше 85000
Швеи на производство спецодежды (Москва) может получать зп больше 85000
Швеи на производство спецодежды (Москва) может получать зп больше 85000
Швея может получать зп больше 85000
Швея - закройщица/закройщик для римских штор может получать зп больше 85000
Швея-портной может получать зп больше 8

26